In [0]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
path_to_file = '/content/drive/My Drive/dataset_1.txt'

In [0]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
vocab = sorted(set(text))

In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [0]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [13]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 187) # (batch_size, sequence_length, vocab_size)


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           47872     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 187)           191675    
Total params: 4,177,851
Trainable params: 4,177,851
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [16]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' the proceedings before an arbitral panel at any time by jointly notifying the chair of the arbitral'

Next Char Predictions: 
 ';2--NΙhpO$Üä”4`λEίπΓή£σ$\n三;r[Br−i〇º−Ö3WΛ>zévwXñb:ΛtÉ£8`u/Εëh’hΝ$œrO[7D«\nη;ó“Rü倍y≥υκά6e三όºÜ.4°NdYόωwF'


In [17]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 187)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.231803


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [20]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
1126/1126 [==============================] - 58s 51ms/step - loss: 1.3813
Epoch 2/10
1126/1126 [==============================] - 58s 51ms/step - loss: 0.8594
Epoch 3/10
1126/1126 [==============================] - 58s 51ms/step - loss: 0.7737
Epoch 4/10
1126/1126 [==============================] - 57s 51ms/step - loss: 0.7279
Epoch 5/10
1126/1126 [==============================] - 57s 51ms/step - loss: 0.7005
Epoch 6/10
1126/1126 [==============================] - 57s 51ms/step - loss: 0.6842
Epoch 7/10
1126/1126 [==============================] - 57s 51ms/step - loss: 0.6755
Epoch 8/10
1126/1126 [==============================] - 57s 51ms/step - loss: 0.6727
Epoch 9/10
1126/1126 [==============================] - 57s 51ms/step - loss: 0.6736
Epoch 10/10
1126/1126 [==============================] - 57s 50ms/step - loss: 0.6792


In [21]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            47872     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 187)            191675    
Total params: 4,177,851
Trainable params: 4,177,851
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [25]:
print(generate_text(model, start_string=u"The government: "))

The government: 
1 

Forespect with a view to ensure that the provisions of this Articles shall reduce the importance of representatives other than as wolly in the avoidance of goods infrastructure will following the date of entry into force of this Agreement the Joint Commity to promote the scope of marks granted in relation to intellectual proper e 




National Treatment), 10.2 (Transfer referred to in this Chapter as would prejudice the legal system. When such measures by a Party from applying measures relating to the entity of a written non-carration of any restrictions specified in determining the qualifying value content of the governmental authority of the exporting Party were required by a situation of the use of a service;   (g) datimume or electronic; and 


(b) 	air application shall be made unless the Parties may agree. 

7. A Party shall not simpley as offered ect to governmental authority of the importing Party may standard, and seef- ensure that its competent authoritie

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
optimizer = tf.keras.optimizers.Adam()

In [0]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [30]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {} PPL {}'
      print(template.format(epoch+1, batch_n, loss, tf.exp(loss)))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f} PPL : {:.6f}'.format(epoch+1, loss, tf.exp(loss)))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 0.713625967502594 PPL 2.041379928588867
Epoch 1 Batch 100 Loss 0.7431025505065918 PPL 2.1024484634399414
Epoch 1 Batch 200 Loss 0.8856378197669983 PPL 2.424530267715454
Epoch 1 Batch 300 Loss 0.8207994103431702 PPL 2.272315740585327
Epoch 1 Batch 400 Loss 0.7856702208518982 PPL 2.1938767433166504
Epoch 1 Batch 500 Loss 0.7744053602218628 PPL 2.169301748275757
Epoch 1 Batch 600 Loss 0.7712295651435852 PPL 2.1624233722686768
Epoch 1 Batch 700 Loss 0.7761399745941162 PPL 2.173068046569824
Epoch 1 Batch 800 Loss 0.6973795890808105 PPL 2.0084826946258545
Epoch 1 Batch 900 Loss 0.6993960738182068 PPL 2.0125370025634766
Epoch 1 Batch 1000 Loss 0.6461818814277649 PPL 1.9082410335540771
Epoch 1 Batch 1100 Loss 0.6841602921485901 PPL 1.9821066856384277
Epoch 1 Loss 0.6485 PPL : 1.912649
Time taken for 1 epoch 55.80367136001587 sec

Epoch 2 Batch 0 Loss 0.6778233051300049 PPL 1.96958589553833
Epoch 2 Batch 100 Loss 0.7087143063545227 PPL 2.0313777923583984
Epoch 2 Batch 200 L